In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1"
import pandas as pd
import pickle
from joblib import Parallel, delayed
import numpy as np
import time
import jax
from jax import devices
import jax.numpy as jnp
import jax.random as jr
from scipy.stats import zscore
import jax.numpy as jnp
import jax.random as jr
from collections import defaultdict
import gc
from dynamax.hidden_markov_model import LinearAutoregressiveHMM, PoissonHMM
from dynamax.hidden_markov_model import PoissonHMM 

# Get my functions
from functions import idxs_from_files, cross_validate_poismodel, cross_validate_armodel, compute_inputs


ERROR:2025-09-22 13:04:31,382:jax._src.xla_bridge:444: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/jax/_src/xla_bridge.py", line 442, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 324, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 281, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: CUDA_ERROR_NO_DEVICE


In [2]:
"""" GRID SEARCH FUNCTIONS """

# Function to perform grid search for a single session
def grid_search_versatile(id, var_interest, zsc, fixed_states, truncate,
                          params, sticky, save_path, data_path, num_train_batches, method, fit_method):
    print(params)
    print(jax.devices())
    # Time job
    start = time.time()
    
    # Load mouse/session data
    mouse_name, session = id
    filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
    original_design_matrix = pd.read_parquet(filename)

    # Zscore if needed (paws)
    if zsc == True:
      array_matrix = zscore(np.array(original_design_matrix), axis=0, nan_policy='omit')
    else:
      array_matrix = np.array(original_design_matrix)

    # Remove NaNs
    filtered_matrix = array_matrix[~np.isnan(array_matrix).any(axis=1)]
    keys = np.where(original_design_matrix.keys().isin(var_interest))
    if truncate:
        design_matrix = filtered_matrix[:100000, keys]  # Truncate session
    else:
        design_matrix = filtered_matrix[:, keys] 
    
    design_matrix = np.reshape(design_matrix,(np.shape(design_matrix)[0], np.shape(design_matrix)[2]))

    fit_id = str(mouse_name + session)

    print(f"Fitting session {fit_id}...")

    num_timesteps = np.shape(design_matrix)[0]
    emission_dim = np.shape(design_matrix)[1]
    shortened_array = np.array(design_matrix[:(num_timesteps // num_train_batches) * num_train_batches])
    train_emissions = jnp.stack(jnp.split(shortened_array, num_train_batches))

    # Initialize result containers
    all_lls, all_baseline_lls = defaultdict(dict), defaultdict(dict)
    all_init_params, all_fit_params = defaultdict(dict), defaultdict(dict)

    if var_interest == ['Lick count']:
        num_states, _, kappas = params
        if fixed_states:
            # Grid search
            for kappa in kappas:
                test_poishmm = PoissonHMM(num_states, emission_dim, transition_matrix_stickiness=kappa)
                all_val_lls, fit_params, init_params, baseline_lls = cross_validate_poismodel(test_poishmm, jr.PRNGKey(0),
                                                                                            train_emissions, num_train_batches, fit_method, num_iters=100)

                all_lls[kappa] = all_val_lls
                all_baseline_lls[kappa] = baseline_lls
                all_init_params[kappa] = init_params
                all_fit_params[kappa] = fit_params
        else:
            for state in num_states:
                # Grid search
                for kappa in kappas:
                    test_poishmm = PoissonHMM(state, emission_dim, transition_matrix_stickiness=kappa)
                    all_val_lls, fit_params, init_params, baseline_lls = cross_validate_poismodel(test_poishmm, jr.PRNGKey(0),
                                                                                               train_emissions, num_train_batches, fit_method, num_iters=100)

                    all_lls[state][kappa] = all_val_lls
                    all_baseline_lls[state][kappa] = baseline_lls
                    all_init_params[state][kappa] = init_params
                    all_fit_params[state][kappa] = fit_params

    else:
        num_states, all_num_lags, kappas = params
        if fixed_states:
            # Grid search
            for lag in all_num_lags:
                for kappa in kappas:
                    test_arhmm = LinearAutoregressiveHMM(num_states, emission_dim, num_lags=lag, transition_matrix_stickiness=kappa)
                    my_inputs = compute_inputs(shortened_array, lag, emission_dim)
                    train_inputs = jnp.stack(jnp.split(my_inputs, num_train_batches))

                    all_val_lls, fit_params, init_params, baseline_lls = cross_validate_armodel(
                        test_arhmm, jr.PRNGKey(0), train_emissions, train_inputs, method, num_train_batches, fit_method
                    )

                    all_lls[lag][kappa] = all_val_lls
                    all_baseline_lls[lag][kappa] = baseline_lls
                    all_init_params[lag][kappa] = init_params
                    all_fit_params[lag][kappa] = fit_params
        else:
            # Grid search
            for state in num_states:
                for lag in all_num_lags:
                    for kappa in kappas:
                        test_arhmm = LinearAutoregressiveHMM(state, emission_dim, num_lags=lag, transition_matrix_stickiness=kappa)
                        my_inputs = compute_inputs(shortened_array, lag, emission_dim)
                        train_inputs = jnp.stack(jnp.split(my_inputs, num_train_batches))

                        all_val_lls, fit_params, init_params, baseline_lls = cross_validate_armodel(
                            test_arhmm, jr.PRNGKey(0), train_emissions, train_inputs, method, num_train_batches, fit_method
                        )

                        all_lls[state][lag][kappa] = all_val_lls
                        all_baseline_lls[state][lag][kappa] = baseline_lls
                        all_init_params[state][lag][kappa] = init_params
                        all_fit_params[state][lag][kappa] = fit_params

    # Save results
    mouse_results = all_lls, all_baseline_lls, all_init_params, all_fit_params, design_matrix, params
    result_filename = os.path.join(save_path, f"{'best_sticky' if sticky else 'best'}_results_{var_interest[0]}_{fit_id}")
    with open(result_filename, "wb") as f:
        pickle.dump(mouse_results, f)

    print(f"Session {fit_id} completed.")
    print("Time:", time.time() - start)
    del mouse_results, all_lls, all_baseline_lls, all_init_params, all_fit_params
    gc.collect()


# Main function for parallel processing
def run_grid_search_parallel(idxs, var_interest, zsc, fixed_states, truncate, params, sticky,
                             save_path, data_path, num_train_batches, method, fit_method, n_jobs):

    # Create folder where to dump data
    if not os.path.exists(save_path):
        # Create a new directory because it does not exist
        os.makedirs(save_path)
   
    # Identify sessions to process
    sessions_to_process = []
    for m, mat in enumerate(idxs):
        mouse_name = mat[37:]
        session = mat[:36]
        fit_id = str(mouse_name + session)
        result_filename = os.path.join(save_path, f"{'best_sticky' if sticky else 'best'}_results_{var_interest[0]}_{fit_id}")
        if not os.path.exists(result_filename):
            sessions_to_process.append((mouse_name, session))
    # sessions_to_process = sessions_to_process

    print(f"Found {len(sessions_to_process)} sessions to process.")

    # Run grid search in parallel
    Parallel(n_jobs=n_jobs)(
        delayed(grid_search_versatile)(
            id, var_interest, zsc,
            fixed_states, truncate, params, sticky, save_path, data_path, num_train_batches, method, fit_method
        ) for id in sessions_to_process
    )

# Main function for parallel processing
def run_grid_search_serial(idxs, var_interest, zsc, fixed_states, truncate, params, sticky,
                             save_path, data_path, num_train_batches, method, fit_method):

    # Create folder where to dump data
    if not os.path.exists(save_path):
        # Create a new directory because it does not exist
        os.makedirs(save_path)
   
    # Identify sessions to process
    sessions_to_process = []
    for m, mat in enumerate(idxs):
        mouse_name = mat[37:]
        session = mat[:36]
        fit_id = str(mouse_name + session)
        result_filename = os.path.join(save_path, f"{'best_sticky' if sticky else 'best'}_results_{var_interest[0]}_{fit_id}")
        if not os.path.exists(result_filename):
            sessions_to_process.append((mouse_name, session))
    # sessions_to_process = sessions_to_process

    print(f"Found {len(sessions_to_process)} sessions to process.")
    
    # Run grid search in series
    for m, mat in enumerate(sessions_to_process):

        grid_search_versatile(
            mat, var_interest, zsc,
            fixed_states, truncate, params, sticky, save_path, data_path, num_train_batches, method, fit_method
        )

In [3]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""""""""""""""""""""""""""""" 'RUN CODE' """"""""""""""""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


prefix = '/home/ines/repositories/representation_learning_variability/'

# Parameters
n_jobs = 2  # Number of CPU cores to use

# LOAD DATA
data_path = prefix + 'paper-individuality/data/design_matrices/'
all_files = os.listdir(data_path)
design_matrices = [item for item in all_files if 'design_matrix' in item and 'standardized' not in item]
idxs, mouse_names = idxs_from_files(design_matrices)

var_interest = ['whisker_me']
# var_interest = ['Lick count']

""" GRID """
all_num_lags = [1, 3, 5, 7, 10, 15, 20, 30]
all_num_lags=list(range(1, 20, 2))
kappas=[0, 0.5, 1, 2, 3, 4, 5, 10, 20, 100]

fixed_states = True
num_states = 2

if var_interest == ['whisker_me']:
    n_jobs = 1
    all_num_lags=[1, 10, 20, 40, 60] 
    all_num_lags=[1, 10, 20, 30] 
    # all_num_lags=list(range(1, 60, 10))
    kappas=[0, 5, 50]
    num_states = 2
elif var_interest == ['Lick count']:
    n_jobs = 3
    kappas=[0, 1, 5, 10, 50, 100, 500, 1000]
    num_states = 2

params = num_states, all_num_lags, kappas

""" FITTING PARAMETERS """
num_train_batches = 5
method='prior'
fit_method='em'
zsc = True
truncate = False

if var_interest == ['whisker_me']:
    zsc = True
elif var_interest == ['Lick count']:
    zsc = False

fitting_params = str(num_train_batches)+'_'+method+'_'+fit_method+'_zsc_'+str(zsc)+'/'
if truncate:
    fitting_params = str(num_train_batches)+'_'+method+'_'+fit_method+'_zsc_'+str(zsc)+'_truncated/'

save_path = prefix + 'paper-individuality/data/hmm/grid_search/'+fitting_params

# # Run the grid search
# run_grid_search_parallel(
#     idxs, var_interest, zsc,
#     fixed_states=fixed_states, truncate=truncate, params=params, sticky=False,
#     save_path=save_path,  data_path=data_path, num_train_batches=num_train_batches, method=method, fit_method=fit_method, n_jobs=n_jobs)

run_grid_search_serial(
    idxs, var_interest, zsc,
    fixed_states=fixed_states, truncate=truncate, params=params, sticky=False,
    save_path=save_path,  data_path=data_path, num_train_batches=num_train_batches, method=method, fit_method=fit_method)

Found 4 sessions to process.
(2, [1, 10, 20, 30], [0, 5, 50])
[CpuDevice(id=0)]
Fitting session UCLA005d57df551-6dcb-4242-9c72-b806cff5613a...
Session UCLA005d57df551-6dcb-4242-9c72-b806cff5613a completed.
Time: 228.4250042438507
(2, [1, 10, 20, 30], [0, 5, 50])
[CpuDevice(id=0)]
Fitting session SWC_0651425bd6f-c625-4f6a-b237-dc5bcfc42c87...
Session SWC_0651425bd6f-c625-4f6a-b237-dc5bcfc42c87 completed.
Time: 248.066241979599
(2, [1, 10, 20, 30], [0, 5, 50])
[CpuDevice(id=0)]
Fitting session UCLA0335455a21c-1be7-4cae-ae8e-8853a8d5f55e...
Session UCLA0335455a21c-1be7-4cae-ae8e-8853a8d5f55e completed.
Time: 211.64462995529175
(2, [1, 10, 20, 30], [0, 5, 50])
[CpuDevice(id=0)]
Fitting session NYU-305569f363-0934-464e-9a5b-77c8e67791a1...
Session NYU-305569f363-0934-464e-9a5b-77c8e67791a1 completed.
Time: 140.54040956497192
